In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.kinopoisk.ru/awards/oscar/2021/' # url страницы
r = requests.get(url)
r.encoding = 'utf-8'  # указываем правильную кодировку принудительно
soup = BeautifulSoup(r.text) #Отправляем полученную страницу в библиотеку для парсинга
tables=soup.find_all('td') #Получаем все таблицы
# tables=soup.find_all('td', {'class': 'news'}) #Получаем все таблицы с вопросами

nom_name_lst_0 = []
nom_name_lst = []

for item in tables:
    if item.get('class') == ['news']:
        nom_name_lst_0.append(item.get_text())
        
for numb, elem in enumerate(nom_name_lst_0):
    if numb%2:
        nom_name_lst.append(elem)
nom_name_lst.pop()

for item in tables:
    if item.get('class') == ['news']:
        nom_name_lst_0.append(item.get_text())

In [2]:
nom_lst_lst = []
ref_lst_lst = []
all_nom = []
all_nom_name = []

tab_ul=soup.find_all('ul')

for tab in tab_ul[:-2]:

    item_lst = []
    film_lst = []
    film_ref = []
    
    for item in tab.children:
        item_lst.append(item)

    for lin in item_lst[1:-1]:
        film_ref.append(lin.find_all('a')[0].get('href'))
        film_lst.append(lin.get_text())
        
    ref_lst_lst.append(film_ref)
    nom_lst_lst.append(film_lst)
    
for nom_name, nom_lst, ref_lst in zip(nom_name_lst, nom_lst_lst, ref_lst_lst):
    for nom in nom_lst:
        nom = nom.split('\n')[0]
        if nom.find('\xa0—\xa0') != -1:
            nom = nom.split('\xa0—\xa0')[1]
        all_nom.append(nom)
        all_nom_name.append(nom_name)

In [12]:
# словарь с номинантами

nom_dict = {}
nom_dict_count = {}

film_sorted = []
film_nom = []

for nom, nom_name in zip(all_nom, all_nom_name):
    if nom_dict.get(nom) == None:
        nom_dict[nom] = []
    nom_dict[nom].append(nom_name)

for nom, nom_name in zip(all_nom, all_nom_name):
    if nom_dict_count.get(nom) == None:
        nom_dict_count[nom] = 0
    nom_dict_count[nom] = nom_dict_count[nom] + 1
    
for i in range(1, 12):
    for film in nom_dict_count:
        if nom_dict_count[film] == i:
            film_sorted.append(film)
film_sorted.reverse()

for i in range(1, 12):
    for film in nom_dict_count:
        if nom_dict_count[film] == i:
            if i == 1:
                film_nom.append(film + ' *(' + nom_dict[film][0] + ')*')
            else:
                film_nom.append(film)
film_nom.reverse()

In [13]:
# создание markdown с фильмами

Line_0 = '## Номинанты на оскар 2021 \n'

file = open("All_films.md", "w", encoding='utf8')

file.write(Line_0)
for film in film_nom:
    file.write('- [ ] ' + str(film) + '\n')
#     file.write('### ' + str(nom_name) + '\n')

file.close()

In [56]:
# создание markdown с номинациями

Line_0 = '# Номинанты на оскар 2021 \n'

file = open("Nominations.md", "w", encoding='utf8')

file.write(Line_0)
for nom_name, nom_lst, ref_lst in zip(nom_name_lst, nom_lst_lst, ref_lst_lst):
    file.write('### ' + str(nom_name) + '\n')
    for nom, ref in zip(nom_lst, ref_lst):
        film_name = nom.split('\n')
        nom_text = '[' + str(film_name[0]) + ' (' + str(film_name[1][:-2]) + ')]'
        ref_text = '(https://www.kinopoisk.ru' + str(ref) + ')'
        file.write('* ' + nom_text + ref_text + '\n')
    file.write('\n')
file.close()

In [64]:
# запись в конец Latex документа с подготовленными настройками с номинациями

Line_1 = 'русский \n'
Line_End = '\end{multicols}\n\end{document}'

file = open("Nominations.tex", "a", encoding='utf8')

for nom_name, nom_lst, ref_lst in zip(nom_name_lst, nom_lst_lst, ref_lst_lst):
    file.write('\section{' + str(nom_name) + '}\n\n')
    file.write(r'\begin{itemize}' + '\n')
               
    for nom, ref in zip(nom_lst, ref_lst):
        film_name = nom.split('\n')
        nom_text = str(film_name[0]) + r'\newline\textit{(' + str(film_name[1][:-2]) + ')}'
        file.write('	\item ' + nom_text + '\n')
            
    file.write('\end{itemize}\n\n')

file.write(Line_End)

file.close()